In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def prophet_forecast(
    series,
    district_name,
    forecast_periods=12,
    yearly_seasonality='auto',
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays=None,
    plot_forecast=True
):
    """
    Prophet time series forecasting with error handling and RMSE calculation.
    Returns (metrics, forecast_df) or (None, None) on error.
    """
    try:
        # 0. Directory setup
        os.makedirs('Prophet_Forecasts', exist_ok=True)

        # 1. Prepare DataFrame for Prophet
        df = pd.DataFrame({'ds': series.index, 'y': series.values})
        df = df.dropna()
        if len(df) < forecast_periods + 12:
            raise ValueError(f"Series too short ({len(df)} points) for {forecast_periods}-step forecast.")

        # 2. Train/test split
        train = df.iloc[:-forecast_periods]
        test = df.iloc[-forecast_periods:]

        # 3. Model setup
        model = Prophet(
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            daily_seasonality=daily_seasonality,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays=holidays
        )
        model.fit(train)

        # 4. Forecast
        future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
        forecast = model.predict(future)

        # 5. Extract forecasted values for test period
        forecast_test = forecast.set_index('ds').loc[test['ds']]
        y_pred = forecast_test['yhat'].values
        y_test = test['y'].values

        # 6. Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        metrics = {
            'district': district_name,
            'rmse': rmse,
            'forecast_periods': forecast_periods
        }

        # 7. Save forecast DataFrame
        forecast_df = pd.DataFrame({
            'district': district_name,
            'date': test['ds'].values,
            'actual': y_test,
            'forecast': y_pred
        })
        forecast_df.to_csv(
            os.path.join('Prophet_Forecasts', f'{district_name}_forecast.csv'),
            index=False
        )

        # 8. Plotting
        if plot_forecast:
            plt.figure(figsize=(12, 6))
            plt.plot(df['ds'], df['y'], label='Original')
            plt.plot(forecast_df['date'], forecast_df['forecast'], label='Prophet Forecast', color='crimson')
            plt.scatter(forecast_df['date'], forecast_df['actual'], label='Actual (Test)', color='black')
            plt.title(f'{district_name} Prophet Forecast (RMSE: {rmse:.2f})')
            plt.legend()
            plt.savefig(os.path.join('Prophet_Forecasts', f'{district_name}_forecast_plot.png'))
            plt.close()

        return metrics, forecast_df

    except Exception as e:
        print(f"⚠️ Skipping {district_name}: {str(e)}")
        return None, None

In [4]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        metrics, forecast = prophet_forecast(
            series=ts,
            district_name=district,
            forecast_periods=6,  # Forecast the last year
            yearly_seasonality='auto',
            weekly_seasonality=False,
            daily_seasonality=False
        )

        
    
        # Show results
        print("=== Metrics ===")
        if metrics:
            print(metrics)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

12:14:09 - cmdstanpy - INFO - Chain [1] start processing
12:14:09 - cmdstanpy - INFO - Chain [1] done processing
12:14:10 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'AHMEDNAGAR', 'rmse': 106.6501375384711, 'forecast_periods': 6}

=== Forecast Data ===
     district       date  actual    forecast
0  AHMEDNAGAR 2020-10-01     343  482.805534
1  AHMEDNAGAR 2020-11-01     468  423.752666
2  AHMEDNAGAR 2020-12-01     305  452.589230
3  AHMEDNAGAR 2021-01-01     372  449.668904
4  AHMEDNAGAR 2021-02-01     301  218.303959
5  AHMEDNAGAR 2021-03-01     316  425.947361


12:14:10 - cmdstanpy - INFO - Chain [1] done processing
12:14:11 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'AKOLA', 'rmse': 156.37399207386676, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual    forecast
0    AKOLA 2020-10-01     493  636.332823
1    AKOLA 2020-11-01     514  758.265970
2    AKOLA 2020-12-01     450  302.838632
3    AKOLA 2021-01-01     481  288.561529
4    AKOLA 2021-02-01     416  497.974632
5    AKOLA 2021-03-01     363  396.134821


12:14:11 - cmdstanpy - INFO - Chain [1] done processing
12:14:12 - cmdstanpy - INFO - Chain [1] start processing


...

12:14:32 - cmdstanpy - INFO - Chain [1] done processing
12:14:32 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'WASHIM', 'rmse': 34.83117651237075, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual    forecast
0   WASHIM 2020-10-01     113  125.591575
1   WASHIM 2020-11-01     154  100.959112
2   WASHIM 2020-12-01     123  166.693059
3   WASHIM 2021-01-01     105  131.821740
4   WASHIM 2021-02-01     100   61.441754
5   WASHIM 2021-03-01      99   85.138033


12:14:32 - cmdstanpy - INFO - Chain [1] done processing


=== Metrics ===
{'district': 'YAVATMAL', 'rmse': 61.44487250223455, 'forecast_periods': 6}

=== Forecast Data ===
   district       date  actual    forecast
0  YAVATMAL 2020-10-01     182  252.315408
1  YAVATMAL 2020-11-01     174  233.365683
2  YAVATMAL 2020-12-01     163  191.676978
3  YAVATMAL 2021-01-01     146  196.666520
4  YAVATMAL 2021-02-01     140  179.917871
5  YAVATMAL 2021-03-01     161  256.923884
⚠️ Skipping MUMBAI SUBURBAN: Series too short (12 points) for 6-step forecast.
=== Metrics ===

=== Forecast Data ===
None


{}

In [5]:
districts

array(['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN'], dtype=object)